In [1]:
import numpy as np
import robosuite as suite

In [18]:
# create environment instance
env = suite.make(
    env_name="Lift",
    robots="Sawyer",
    has_renderer=False,
    has_offscreen_renderer=False,
    use_camera_obs=False,
)

env.reset()
print()

### Observations from the environment (Sawyer)
- joint position          | robot0_joint_pos (7 dim)
- joint velocity          | robot0_joint_vel (7 dim)
- end effector position   | robot0_eef_pos   (3 dim)
- end effector rotation   | robot0_eef_quat (4 dim)
- gripper position        | robot0_gripper_qpos (2 dim)
- gripper velocity        | robot0_gripper_qvel (2 dim)
- overall robot state     | robot0_robot-state - (32 dim) ref(single-arm.py) (depends on the robotic arm)
    1. sin(robot0_joint_pos) (7 dim)
    2. cos(robot0_joint_pos) (7 dim)
    3. other robot states (18 dim)
- cube_pos
- cube_quat
- robot0_gripper_to_cube
- object-state (10 dim) ref(lift.py) (depends on the chosen environment/task)
    1. cube_pos (3 dim)
    2. cube_quat (rotation) (4 dim)
    3. robot0_gripper_to_cube (3 dim)

### Rewards
- Scalar reward

### Done
- True/False

In [19]:
for i in range(1):
    action = np.random.randn(env.robots[0].dof) # sample random action
    print(action)
    obs, reward, done, info = env.step(action) # take action in the environment
#     print(f'observation: {(obs)}\n')
#     print(f'reward: {reward}\n')
#     print(f'done: {done}\n')
#     print(f'info: {info}\n')

[-0.37891592  0.03030201  0.43327661 -0.79140415 -0.0973005   1.72768553
 -0.59457935  1.21270954]


In [20]:
len(obs['object-state']) + len(obs['robot0_robot-state'])

42

In [24]:
#obs['robot0_robot-state'], obs

### Gym environment support

In [6]:
from robosuite.wrappers import GymWrapper

In [7]:
gym_env = GymWrapper(
    suite.make(
        "Lift",
        robots="Sawyer",
        use_camera_obs=False,
        has_offscreen_renderer=False,
        has_renderer=False,
        reward_shaping=True,
        control_freq=20,
    )
)

adding key: robot0_robot-state
adding key: object-state


C:\Users\91953\Anaconda3\envs\pfrl\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [8]:
#gym_env.action_space
action = gym_env.action_space.sample()
obs, reward, done, info = gym_env.step(action)

In [25]:
gym_env.observation_space # 42 dimensions, it contains the robot-state + object-state from above

Box(-inf, inf, (42,), float32)

In [28]:
# 8 dimensions, it contains action_limits = robot degrees of freedom (Sawyer - 7 dim) + controller control limits (1 dim)
# ref(osc.py)
#gym_env.action_space 
action

array([-0.37891592,  0.03030201,  0.43327661, -0.79140415, -0.0973005 ,
        1.72768553, -0.59457935,  1.21270954])

In [12]:
obs

array([ 0.02708954, -0.9103308 , -0.01245366,  0.80267952,  0.00729927,
        0.56043663, -0.15060884,  0.99963301,  0.41388142,  0.99992245,
       -0.59641059,  0.99997336,  0.82819731, -0.98859343,  0.00695299,
       -0.07916572,  0.00192717,  0.04557523, -0.01784412, -0.20496173,
        0.1323188 , -0.14567943,  0.16757051,  1.00268517, -0.69468785,
       -0.71789391,  0.03225366,  0.0315727 ,  0.01090604, -0.01087996,
       -0.19973008,  0.20027009, -0.0248905 ,  0.02095832,  0.81859601,
        0.        ,  0.        ,  0.99779415,  0.06638394, -0.12078893,
        0.14661218,  0.18408915])

In [3]:
import torch

In [5]:
torch.cuda.get_device_name(0)

'GeForce GTX 1660 Ti'

In [ ]:
for i_episode in range(20):
    observation = env.reset()
    for t in range(500):
        env.render()
        action = env.action_space.sample()